In [1]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from html_table_parser import parser_functions
import pandas as pd

In [2]:
import collections
collections.Callable = collections.abc.Callable

In [3]:
# 드라이버 초기화
driver = webdriver.Chrome()


try:
    columns = pd.MultiIndex.from_tuples([
    ('', '날짜'),
    ('', '종가'),
    ('', '전일비'),
    ('', '등락률'),
    ('', '거래량'),
    ('기관', '순매매량'),
    ('외국인', '순매매량'),
    ('외국인', '보유주수'),
    ('외국인', '보유율')
    ])
    df = pd.DataFrame(columns=columns)
    for i in range(1,244):
        req_url = 'https://finance.naver.com/item/frgn.naver?code=005930&page='+str(i)
        driver.get(req_url)
        # 페이지의 HTML 소스를 가져와서 파싱
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.find("table", {"summary": "외국인 기관 순매매 거래량에 관한표이며 날짜별로 정보를 제공합니다."})
        table = parser_functions.make2d(table)
        frame = pd.DataFrame(data = table[1:], columns = table[0])
        frame = pd.DataFrame(data=table[1:], columns=table[0])
        frame.columns = pd.MultiIndex.from_tuples([('', '날짜'),
                                                   ('', '종가'),
                                                   ('', '전일비'),
                                                   ('', '등락률'),
                                                   ('', '거래량'),
                                                   ('기관', '순매매량'),
                                                   ('외국인', '순매매량'),
                                                   ('외국인', '보유주수'),
                                                   ('외국인', '보유율')
                                                    ])

        # 만약 '외국인 보유율' 컬럼이 table 데이터에 없다면 이를 추가
        if ('외국인', '보유율') not in frame.columns:
            frame[('', '외국인 보유율')] = None

        # 기존 df와 새로운 frame 데이터프레임을 병합
        df = pd.concat([df, frame], axis=0, ignore_index=True)

finally:
    driver.quit()  # 드라이버 종료

In [4]:
df = df.iloc[2:,:]

In [5]:
df = df.drop_duplicates()

In [6]:
# 인덱스가 7, 32, 33이 아닌 행만 필터링
filtered_df = df[~df.index.isin([7, 32, 33])]

In [7]:
filtered_df.head(50)

기관  \
            날짜      종가                전일비      등락률         거래량        순매매량   
2   2024.08.21  78,300    하락\n\t\t\t\t600   -0.76%   7,734,771    +177,643   
3   2024.08.20  78,900    상승\n\t\t\t\t600   +0.77%  10,683,836    -141,999   
4   2024.08.19  78,300  하락\n\t\t\t\t1,900   -2.37%  14,146,565    -339,031   
5   2024.08.16  80,200  상승\n\t\t\t\t3,000   +3.89%  22,061,478    +918,078   
6   2024.08.14  77,200  상승\n\t\t\t\t1,100   +1.45%  13,246,168    -230,603   
10  2024.08.13  76,100    상승\n\t\t\t\t600   +0.79%  10,716,261    -202,595   
11  2024.08.12  75,500    상승\n\t\t\t\t800   +1.07%   9,839,259    +112,759   
12  2024.08.09  74,700  상승\n\t\t\t\t1,300   +1.77%  16,388,222     -21,816   
13  2024.08.08  73,400  하락\n\t\t\t\t1,300   -1.74%  28,414,729  -3,828,398   
14  2024.08.07  74,700  상승\n\t\t\t\t2,200   +3.03%  32,710,428  -2,123,457   
18  2024.08.06  72,500  상승\n\t\t\t\t1,100   +1.54%  47,295,226  -7,018,788   
19  2024.08.05  71,400  하락\n\t\t\t\t8,200  -10.30%  54,608,790  -2,439,089   
20  2024.08.02  79,600  하락\n\t\t\t\t3,500   -4.21%  25,800,275  -3,138,554   
21  2024.08.01  83,100    하락\n\t\t\t\t800   -0.95%  20,900,338  -1,061,536   
22  2024.07.31  83,900  상승\n\t\t\t\t2,900   +3.58%  20,744,323    +958,338   
26  2024.07.30  81,000    하락\n\t\t\t\t200   -0.25%  13,169,636  -1,379,203   
27  2024.07.29  81,200    상승\n\t\t\t\t300   +0.37%  12,797,136    -554,806   
28  2024.07.26  80,900    상승\n\t\t\t\t500   +0.62%  14,508,334    +719,026   
29  2024.07.25  80,400  하락\n\t\t\t\t1,600   -1.95%  20,323,811  -1,877,734   
30  2024.07.24  82,000  하락\n\t\t\t\t1,900   -2.26%  16,939,083    -492,483   
34  2024.07.23  83,900    상승\n\t\t\t\t900   +1.08%  15,766,389  -1,694,295   
35  2024.07.22  83,000  하락\n\t\t\t\t1,400   -1.66%  18,987,560    -242,867   
36  2024.07.19  84,400  하락\n\t\t\t\t2,500   -2.88%  18,569,122  -1,750,534   
37  2024.07.18  86,900    상승\n\t\t\t\t200   +0.23%  24,721,790  +1,853,095   
38  2024.07.17  86,700  하락\n\t\t\t\t1,000   -1.14%  18,186,490    -306,635   
42  2024.07.16  87,700  상승\n\t\t\t\t1,000   +1.15%  16,166,688    +645,166   
43  2024.07.15  86,700  상승\n\t\t\t\t2,300   +2.73%  25,193,080  +4,178,194   
44  2024.07.12  84,400  하락\n\t\t\t\t3,200   -3.65%  26,344,386    +154,761   
45  2024.07.11  87,600    하락\n\t\t\t\t200   -0.23%  24,677,608  -1,171,760   
46  2024.07.10  87,800                보합0    0.00%  17,813,847  -1,447,489   
50  2024.07.09  87,800    상승\n\t\t\t\t400   +0.46%  21,336,201      -3,751   
51  2024.07.08  87,400    상승\n\t\t\t\t300   +0.34%  24,035,809  -1,128,290   
52  2024.07.05  87,100  상승\n\t\t\t\t2,500   +2.96%  45,791,193  +6,809,451   
53  2024.07.04  84,600  상승\n\t\t\t\t2,800   +3.42%  43,857,229  +6,993,695   
54  2024.07.03  81,800                보합0    0.00%  11,440,328    +366,480   
58  2024.07.02  81,800                보합0    0.00%  14,471,904  -1,172,626   
59  2024.07.01  81,800    상승\n\t\t\t\t300   +0.37%  11,317,202    +442,996   
60  2024.06.28  81,500    하락\n\t\t\t\t100   -0.12%   9,455,929    +207,314   
61  2024.06.27  81,600    상승\n\t\t\t\t300   +0.37%  11,739,720  -3,472,885   
62  2024.06.26  81,300    상승\n\t\t\t\t500   +0.62%  17,783,242    -793,488   
66  2024.06.25  80,800    상승\n\t\t\t\t200   +0.25%  19,088,458    -476,258   
67  2024.06.24  80,600    상승\n\t\t\t\t600   +0.75%  15,454,227    +281,038   
68  2024.06.21  80,000  하락\n\t\t\t\t1,600   -1.96%  17,907,523    -392,128   
69  2024.06.20  81,600    상승\n\t\t\t\t400   +0.49%  20,288,913  +1,958,303   
70  2024.06.19  81,200  상승\n\t\t\t\t1,400   +1.75%  24,168,863  +3,843,571   
74  2024.06.18  79,800  상승\n\t\t\t\t1,700   +2.18%  18,207,598    +401,243   
75  2024.06.17  78,100  하락\n\t\t\t\t1,500   -1.88%  16,123,051    -947,913   
76  2024.06.14  79,600  상승\n\t\t\t\t1,000   +1.27%  22,926,612  -1,159,458   
77  2024.06.13  78,600  상승\n\t\t\t\t2,100   +2.75%  33,763,804    -350,325   
78  2024.06.12  76,500  상승\n\t\t\t\t1,300   +1.73%  19,246,725     -75,724   

            외국인

In [8]:
import re

def convert_to_number(value):
    # 문자열에서 '하락' 또는 '상승'을 찾아서 부호 결정
    if '하락' in value:
        sign = -1
    elif '상승' in value:
        sign = 1
    elif '보합' in value:
        sign = 0
    else:
        raise ValueError("Unrecognized pattern in value")
    
    # 숫자 부분만 추출하기 위해 정규표현식을 사용
    number_str = re.search(r'[\d,]+', value).group()
    
    # 쉼표 제거 후 정수로 변환
    number = int(number_str.replace(',', ''))
    
    # 부호를 적용한 값 반환
    return sign * number

# 예제 사용
example_str = '하락\n\t\t\t\t600'
converted_value = convert_to_number(example_str)
print(converted_value)  # 출력: -600

-600


In [9]:
filtered_df[('', '전일비')] = filtered_df[('', '전일비')].apply(lambda x : convert_to_number(x))

C:\Users\wodnj\AppData\Local\Temp\ipykernel_63288\3424287303.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[('', '전일비')] = filtered_df[('', '전일비')].apply(lambda x : convert_to_number(x))


In [10]:
filtered_df

기관         외국인  \
              날짜       종가   전일비     등락률         거래량      순매매량        순매매량   
2     2024.08.21   78,300  -600  -0.76%   7,734,771  +177,643     +82,341   
3     2024.08.20   78,900   600  +0.77%  10,683,836  -141,999    +924,485   
4     2024.08.19   78,300 -1900  -2.37%  14,146,565  -339,031  -1,468,641   
5     2024.08.16   80,200  3000  +3.89%  22,061,478  +918,078  +6,540,655   
6     2024.08.14   77,200  1100  +1.45%  13,246,168  -230,603  +2,054,899   
...          ...      ...   ...     ...         ...       ...         ...   
7758  2005.01.07  440,500  5500  +1.26%     272,369   -73,660     +71,587   
7762  2005.01.06  435,000 -8000  -1.81%     343,063   -56,059      +3,706   
7763  2005.01.05  443,000 -4000  -0.89%     467,322    -9,961      +1,683   
7764  2005.01.04  447,000 -4000  -0.89%     280,347   -31,824     +18,352   
7765  2005.01.03  451,000   500  +0.11%     218,558   +32,006      -6,036   

                             
               보유주수     보유율  
2     3,357,989,044  56.25%  
3     3,357,906,703  56.25%  
4     3,356,978,020  56.23%  
5     3,358,563,961  56.26%  
6     3,352,025,208  56.15%  
...             ...     ...  
7758     79,831,619  54.20%  
7762     79,760,032  54.15%  
7763     79,752,966  54.14%  
7764     79,751,333  54.14%  
7765     79,732,931  54.13%  

[4854 rows x 9 columns]

In [11]:
filtered_df.to_csv('투자자별 매매동향.csv', index = False , encoding = 'utf-8-sig')